# Importance 11 Subdistricts: ngothana_process

### Importance11 Subdistricts: ngothana_process
- DONE: Step 1: List all files and check if empty. Subset based on emptyness
- DONE: Step 2: Read file and add file name, split file name to additional columns and add column with path
- DONE: Step 3: Define variables and if necessary to calculate relative values
- DONE: Step 4: Determine how to sumarize columns if possible to sum or average and do so
- DONE: Step 5: Break data by years
- DONE: Step 6: Summarize data by zilas. Keep numeric variables, division, zila and year
- DONE: Step 7: Create file with zilas, division and 
- DONE: Step 8: Write everything to the output files

- Variables:
    - **thana**:
    - **ECouple**: 
    - **Pill**:
    - **Condom**:
    - **Injectable**: 
    - **IUD**: 
    - **Implant**:
    - **PerMale**: 
    - **PerFemale**:
    - **Tpermanent**: 
    - **GrandTotal**:
    - **CAR**:
    - **Unnamed: 12**:
    - **full_name_path**: 
    - **file_name1**: 
    - **file_name2**:
    - **year**: 
    - **month**: 
    - **upazila**: 
    - **division**: 
    - **upazila_full**:
- Indicators:
    - Percent_Pill = Pill/GrandTotal
    - Percent_Condom = Condom/GrandTotal
    - Percent_Injectable = Injectable/GrandTotal
    - Percent_IUD = IUD/GrandTotal
    - Percent_Implant = Implant/GrandTotal
    - Percent_PerMale = PerMale/Tpermanent
    - Percent_PerFemale = PerFemale/Tpermanent
    - CAR

## Paths

In [69]:
DATA = '/Users/edinhamzic/Symphony/wb_bangladesh/Bangladesh/data/dgfp/data/importance11_subdistricts/ngothana_process'
GEO = '/Users/edinhamzic/Symphony/wb_bangladesh/Bangladesh/output/dgfp/geos/dgfp_geo.csv'
OUT = '/Users/edinhamzic/Symphony/wb_bangladesh/Bangladesh/output/dgfp/data/'

## Packages

In [2]:
import os
import re
import sys
import glob
import numpy as np
import pandas as pd

## Custom functions

In [3]:
def list_files(path):
    files_list = []
    for path, subdirs, files in os.walk(path):
        for file in files:
            if bool(re.search(pattern=r'.csv', string=file)):
                files_list.append(os.path.join(path, file))
    return files_list
    
def read_arrange_files(files_list):
    data_list = {}
    log_columns = ['empty', 'full_name_path', 'file_name1', 'file_name2',
                   'year', 'month', 'upazila', 'division', 'upazila_full']
    log_array = []
    for file in files_list:
        tmp = pd.read_csv(file, sep='\t', skiprows=1)
        log_list = []
        log_list.append(tmp.empty)
        if tmp.empty:
            log_list.append(file)
            log_list.append(os.path.split(file)[1])
            name = os.path.split(file)[1]
            name = name.replace(".csv", "")
            log_list.append(name)
            name_split = name.split("-")
            log_list.append(name_split[0])
            log_list.append(name_split[1])
            log_list.append(name_split[3])
            log_list.append(name_split[4])
            log_list.append(name_split[4]+name_split[3])
        else:
            tmp['full_name_path'] = file
            log_list.append(file)
            tmp['file_name1'] = os.path.split(file)[1]
            log_list.append(os.path.split(file)[1])
            name = os.path.split(file)[1]
            name = name.replace(".csv", "")
            tmp['file_name2'] = name
            log_list.append(name)
            name_split = name.split("-")
            tmp['year'] = int(name_split[0])
            log_list.append(name_split[0])
            tmp['month'] = int(name_split[1])
            log_list.append(name_split[1])
            tmp['upazila'] = name_split[3]
            log_list.append(name_split[3])
            tmp['division'] = name_split[4]
            log_list.append(name_split[4])
            tmp['upazila_full'] = name_split[4]+name_split[3]
            log_list.append(name_split[4]+name_split[3])
            data_list[name] = tmp
        log_array.append(log_list)
    return data_list, pd.DataFrame(log_array, columns=log_columns)

def concat_data(ddict):
    out_list = []
    for key, value in ddict.items():
        out_list.append(value)
    df = pd.concat(out_list, axis=0)
    return df

def break_by_years(input_df, year_var):
    df = input_df.copy(deep=True)
    out_dict = {}
    for year in df[year_var].unique():
        out_dict[str(year)] = df[df[year_var] == year]
    return out_dict

def summarize_zilas(data_dict):
    out_dict = {}
    for key, value in data_dict.items():
        tmp = value.copy(deep=True)
        subset_vars = ['upazila', 'division', 'ECouple', 'Pill', 'Condom', 'Injectable', 'IUD',
                       'Implant','PerMale', 'PerFemale', 'Tpermanent', 'GrandTotal']
        tmp = tmp[subset_vars]
        tmp = tmp.groupby('upazila').sum()
        tmp = tmp.reset_index()
        out_dict[key] = tmp
    return out_dict


def calculate_indicators(data_dict):
    out_dict = {}
    for key, value in data_dict.items():
        tmp = value.copy(deep=True)
        tmp['NGO_Percent_Pill'] = np.round(tmp['Pill']/tmp['GrandTotal']*100, 2)
        tmp['NGO_Percent_Condom'] = np.round(tmp['Condom']/tmp['GrandTotal']*100, 2)
        tmp['NGO_Percent_Injectable'] = np.round(tmp['Injectable']/tmp['GrandTotal']*100, 2)
        tmp['NGO_Percent_IUD'] = np.round(tmp['IUD']/tmp['GrandTotal']*100, 2)
        tmp['NGO_Percent_Implant'] = np.round(tmp['Implant']/tmp['GrandTotal']*100, 2)
        tmp['NGO_Percent_PerMale'] = np.round(tmp['PerMale']/tmp['GrandTotal']*100, 2)
        tmp['NGO_Percent_PerFemale'] = np.round(tmp['PerFemale']/tmp['GrandTotal']*100, 2)
        tmp['NGO_CAR'] = np.round(tmp['GrandTotal']/tmp['ECouple']*100, 2)
        tmp = tmp[['upazila', 'NGO_Percent_Pill', 'NGO_Percent_Condom', 'NGO_Percent_Injectable', 
                   'NGO_Percent_IUD', 'NGO_Percent_Implant', 'NGO_Percent_PerMale',
                   'NGO_Percent_PerFemale', 'NGO_CAR',]]
        out_dict[key] = tmp
    return out_dict

def update_geos(datad, geo_df):
    out = {}
    for key, value in datad.items():
        tmp = geo_df.merge(value, how='left', left_on='upazila_dgfp', right_on='upazila')
        out[key] = tmp
    return out


def clean_up(data_dict):
    out = {}
    for key, value in data_dict.items():
        tmp = value
        tmp['geo'] = tmp['division_geo'].str.cat(tmp['zila_geo'], sep="")
        tmp = value.drop(['upazila_dgfp', 'division_dgfp', 'upazila_full_dgfp', 'division_geo',
                          'division', 'zila_geo','zila','upazila'], axis=1)
        out[key] = tmp
    return out

## Data reading and processing 

In [4]:
files = list_files(path=DATA) 
data, log = read_arrange_files(files_list=files)

### Log data

In [5]:
log['empty'].value_counts()
log_true = log[log['empty'] == True]
display(log_true.head())

,empty,full_name_path,file_name1,file_name2,year,month,upazila,division,upazila_full
2,True,/Users/edinhamzic/Symphony/wb_bangladesh/Bangl...,2010-4-28-0602-06.csv,2010-4-28-0602-06,2010,4,0602,06,060602
3,True,/Users/edinhamzic/Symphony/wb_bangladesh/Bangl...,2013-9-28-0304-03.csv,2013-9-28-0304-03,2013,9,0304,03,030304
13,True,/Users/edinhamzic/Symphony/wb_bangladesh/Bangl...,2014-5-28-0106-07.csv,2014-5-28-0106-07,2014,5,0106,07,070106
18,True,/Users/edinhamzic/Symphony/wb_bangladesh/Bangl...,2014-11-28-0109-01.csv,2014-11-28-0109-01,2014,11,0109,01,010109
22,True,/Users/edinhamzic/Symphony/wb_bangladesh/Bangl...,2018-10-28-0507-05.csv,2018-10-28-0507-05,2018,10,0507,05,050507


## Missing data by years

In [6]:
print(log_true['year'].value_counts())

2015    243
2018    212
2013    189
2014    186
2012    170
2008    104
2011     89
2009     65
2010     56
2017     11
2016      8
Name: year, dtype: int64


## Missing data by months

In [7]:
print(log_true['month'].value_counts())

10    159
11    151
12    148
5     117
2     115
6      98
9      95
4      93
7      93
3      91
8      87
1      86
Name: month, dtype: int64


## Missing data by divisions

In [8]:
print(log_true['division'].value_counts())

05    258
07    226
06    222
04    174
02    169
01    128
08     82
03     74
Name: division, dtype: int64


In [9]:
data_df = concat_data(ddict=data)
print(data_df.shape)

(29260, 21)


In [10]:
display(data_df.head())
data_df.columns

,thana,ECouple,Pill,Condom,Injectable,IUD,Implant,PerMale,PerFemale,Tpermanent,...,CAR,Unnamed: 12,full_name_path,file_name1,file_name2,year,month,upazila,division,upazila_full
0,Panchgarh Sadar,8695,3479,842,1265,116,368,218,679,897,...,80.1265,NaN,/Users/edinhamzic/Symphony/wb_bangladesh/Bangl...,2012-9-28-0103-07.csv,2012-9-28-0103-07,2012,9,0103,07,070103
1,Atowari,9493,3363,274,1555,401,266,774,737,1511,...,77.6362,NaN,/Users/edinhamzic/Symphony/wb_bangladesh/Bangl...,2012-9-28-0103-07.csv,2012-9-28-0103-07,2012,9,0103,07,070103
2,Debiganj,11190,3979,283,3035,211,95,740,287,1027,...,77.1224,NaN,/Users/edinhamzic/Symphony/wb_bangladesh/Bangl...,2012-9-28-0103-07.csv,2012-9-28-0103-07,2012,9,0103,07,070103
0,Moheshpur,14440,4864,930,3234,652,410,81,1249,1330,...,79.0859,NaN,/Users/edinhamzic/Symphony/wb_bangladesh/Bangl...,2013-1-28-0204-02.csv,2013-1-28-0204-02,2013,1,0204,02,020204
1,Jhenaidaha Sadar,16230,6735,1633,2844,196,229,65,658,723,...,76.1553,NaN,/Users/edinhamzic/Symphony/wb_bangladesh/Bangl...,2013-1-28-0204-02.csv,2013-1-28-0204-02,2013,1,0204,02,020204


Index(['thana', 'ECouple', 'Pill', 'Condom', 'Injectable', 'IUD', 'Implant',
       'PerMale', 'PerFemale', 'Tpermanent', 'GrandTotal', 'CAR',
       'Unnamed: 12', 'full_name_path', 'file_name1', 'file_name2', 'year',
       'month', 'upazila', 'division', 'upazila_full'],
      dtype='object')

## Break data by years

In [11]:
datad = break_by_years(input_df=data_df, year_var='year')
datad.keys()

dict_keys(['2012', '2013', '2011', '2008', '2018', '2016', '2017', '2010', '2015', '2009', '2014'])

## Summarize data 

In [59]:
data = summarize_zilas(data_dict=datad)
data = calculate_indicators(data_dict=data)
for key, value in data.items():
    print(key)
    print(value.shape)

2012
(62, 9)
2013
(55, 9)
2011
(60, 9)
2008
(60, 9)
2018
(62, 9)
2016
(64, 9)
2017
(64, 9)
2010
(61, 9)
2015
(55, 9)
2009
(61, 9)
2014
(58, 9)


## Read geographies

In [60]:
dgfp_geo = pd.read_csv(GEO)
dgfp_geo['upazila_dgfp'] = dgfp_geo['upazila_dgfp'].astype(str).str.pad(width=4, side='left', fillchar='0')
dgfp_geo['division_dgfp'] = dgfp_geo['division_dgfp'].astype(str).str.pad(width=2, side='left', fillchar='0')
dgfp_geo['upazila_full_dgfp'] = dgfp_geo['upazila_full_dgfp'].astype(str).str.pad(width=6, side='left', fillchar='0')
dgfp_geo['division_geo'] = dgfp_geo['division_geo'].astype(str).str.pad(width=2, side='left', fillchar='0')
dgfp_geo['zila_geo'] = dgfp_geo['zila_geo'].astype(str).str.pad(width=2, side='left', fillchar='0')
dgfp_geo = dgfp_geo.drop(['thana_dgfp', 'Unnamed: 8'], axis=1)
print(dgfp_geo.shape)
dgfp_geo.head()
dgfp_geo = dgfp_geo.drop_duplicates()
print(dgfp_geo.shape)

(29260, 7)
(64, 7)


## Update geographies

In [61]:
data = update_geos(datad=data, geo_df=dgfp_geo)

2012
(62, 9)
(64, 16)
2013
(55, 9)
(64, 16)
2011
(60, 9)
(64, 16)
2008
(60, 9)
(64, 16)
2018
(62, 9)
(64, 16)
2016
(64, 9)
(64, 16)
2017
(64, 9)
(64, 16)
2010
(61, 9)
(64, 16)
2015
(55, 9)
(64, 16)
2009
(61, 9)
(64, 16)
2014
(58, 9)
(64, 16)


## Clean up before writing data

In [65]:
data = clean_up(data_dict=data)

## Writing data

In [79]:
def write_out(data_dict, out_dir):
    out = {}
    for key, value in data_dict.items():
        value.to_csv(path_or_buf=os.path.join(out_dir, "data_dgfp_imp11subdistr_ngothanaprocess_" + str(key) + ".csv"), 
                    index=False, index_label=False)
write_out(data_dict=data, out_dir=OUT)

## Deprecated code

```
dgfp_imp11subdist_years = break_by_years(data_dict=dgfp_imp11subdist, start_year=2008, end_year=2018, year_var='Year')
dgfp_imp11subdist_data = summarize_data(data_dict = dgfp_imp11subdist_years, index_vars=['thana_Name', 'thana', 'Year', 'District', 'Division', 'program'])
data = merge_by_years(dgfp_imp11subdist_data)  
def dgfp_geos(data_dict, geo_vars):
    out = {}
    for key, value in data_dict.items():
        subset = list(set(geo_vars).intersection(set(value.columns)))
        tmp = value[subset]
        tmp = tmp.drop_duplicates()
        out[key] = tmp
    return out
        
t = dgfp_geos(data_dict=dgfp_imp11subdist, geo_vars = ['thana_Name', 'thana', 'District', 'Division', 'program'])
for k,v in t.items():
    print("#"*100)
    print(k)
    print(v.shape)
    display(v.head())
t['../../data/dgfp/data/importance11_subdistricts/thana_process'].to_csv("../../dgfp_geo.csv")
def merge_by_years(data_dict):
    out = {}
    for year, data in data_dict.items():
        out[year] = pd.concat([v for k,v in data.items()], axis=1, sort=True, join='outer',)
    return out

def read_data_dir(data_dirs, ext):
    data_dict= {}
    dirs = pd.read_csv(data_dirs, header=None, names=['dirs'])
    for folder in dirs['dirs']:
        # print(folder)
        data_dict[folder] = read_data(path=folder, ext=ext)
    return data_dict
    
def summarize_data(data_dict, index_vars):
    years_dict = {}
    for key, value in data_dict.items():
        years_dict[key] = {}
        for program, df in value.items():
            tmp_index = list(set(index_vars).intersection(set(df.columns)))
            df = df.set_index(tmp_index)
            df = df[list(df.dtypes[df.dtypes == 'float64'].index)]
            try:
                df = df.groupby(['thana']).sum()
            except KeyError:
                df = df.groupby(['thana_Name']).sum()
            df.columns = [var + "_" + program.split("/")[-1] for var in df.columns]
            years_dict[key][program] = df
    return years_dict 
```